In [2]:
#import gensim.function

text=open('Стихи.txt','r').readlines()
text = [i for i in text if i != '' and 'Стихотворения' not in i]

In [3]:
text = [i.strip() for i in text]

In [4]:
text = [i for i in text if i != '' and 'Стихотворения' not in i]

In [11]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
len(text)

22121

In [7]:
text = text[:len(text)-243]

In [8]:
data=''.join(text)

In [9]:
len(data)

603368

In [12]:
symbols = 'АБВГДЕЁЖЗИЙКЛМПОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
print(len(symbols))
data = (''.join([i for i in data if i in symbols]))

98


509861